# store

In [ ]:
config_items = get_claude_config_items()

# TODO
**list mcp servers needed for demo**:
- github
- custom notes mcp over streamable http?
- screenpipe (only download )


**design daemon framework**:
- skip first, just check before every command if
- 1 there were manual changes to mcp servers (for stdio, proxies)
- 

**toolbox daemon responsibilities:**
- start at startup
- for all local http servers, make sure they are on
- perhaps update the state? what if someone manually installed an mcp in cursor




- implement installation for demo mcp servers
- implement daemon such that they are always online

In [ ]:
installed(conn)

In [ ]:
install(name="github mcp", clients=["claude"])

In [ ]:
installed(conn)

In [ ]:
install(name="todo extractor mcp", clients=["claude"])

In [ ]:
installed(conn)
